# Feature Engineering

## Importing Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/DSA4262Proj2

/content/drive/MyDrive/DSA4262Proj2


## Parsing JSON File

In [3]:
import json
import numpy as np
import pandas as pd

data = [json.loads(line) for line in open('data.json','r')]
info = pd.read_csv('data.info')

In [4]:
res = []
for row in data:
    for trans_id in row.keys():
        for trans_pos in row[trans_id].keys():
            for nucleo_seq in row[trans_id][trans_pos].keys():
                temp = list(np.mean(np.array(row[trans_id][trans_pos][nucleo_seq]), axis=0))
                res.append([trans_id, trans_pos, nucleo_seq] + temp)
print(res)
data = pd.DataFrame(res, columns = ['transcript_id', 'transcript_pos', 'nucleo_seq',
                                  'dwell_time_-1', 'sd_-1', 'mean_-1',
                                  'dwell_time_0', 'sd_0', 'mean_0',
                                  'dwell_time_1', 'sd_1', 'mean_1'
                                 ])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
data['transcript_pos'] = data.transcript_pos.astype(int)
new_df = pd.merge(data, info, left_on=['transcript_id','transcript_pos'],right_on=['transcript_id','transcript_position'], how = 'left')
new_df = new_df.sort_values(by=['gene_id'], ascending=False).reset_index(drop=True)

In [6]:
labels = new_df.label

In [7]:
new_df

,transcript_id,transcript_pos,nucleo_seq,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,sd_1,mean_1,gene_id,transcript_position,label
0,ENST00000641784,2338,CAGACAT,0.008901,5.516981,108.167925,0.009253,6.466415,127.094340,0.007320,3.799811,87.543396,ENSG00000284707,2338,0
1,ENST00000641784,1822,CAAACAA,0.008753,2.364118,104.647059,0.006587,2.839725,99.323529,0.007438,2.130980,88.862745,ENSG00000284707,1822,0
2,ENST00000641390,1698,ATGACAG,0.007056,4.300926,87.257407,0.006359,9.203148,114.444444,0.007459,4.007407,82.725926,ENSG00000284707,1698,0
3,ENST00000641390,1651,AGAACAA,0.006502,9.992264,125.886792,0.007257,3.557170,95.435849,0.006305,2.351472,87.858491,ENSG00000284707,1651,0
4,ENST00000641390,1641,GAAACTG,0.006711,2.609642,110.018868,0.006727,2.863396,107.941509,0.007009,2.627660,95.028302,ENSG00000284707,1641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000373020,830,ATAACCA,0.007765,2.834000,86.765000,0.007522,3.192500,93.075000,0.005923,2.020500,80.110000,ENSG00000000003,830,0
121834,ENST00000373020,823,ATAACAA,0.007456,2.078095,86.904762,0.006193,1.912857,93.680952,0.007296,2.306667,88.176190,ENSG00000000003,823,0
121835,ENST00000373020,689,TAAACAA,0.009868,1.972857,102.495238,0.007332,2.551905,97.928571,0.006917,2.198571,89.385714,ENSG00000000003,689,0
121836,ENST00000373020,512,ATAACTC,0.007247,2.359500,86.795000,0.011178,2.496500,97.965000,0.008500,1.588750,92.330000,ENSG00000000003,512,0


## Feature Engineering

### FNV Hashing and MurmurHash

In [8]:
def getKmers(sequence, size):
    return [sequence[x:x+size] for x in range(len(sequence) - size + 1)]

In [9]:
!pip install mmh3
!pip install screed
!pip install pyhash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import mmh3
import screed
import pyhash

def murmur_hash(kmer):
    # calculate the reverse complement
    rc_kmer = screed.rc(kmer)
    #print(rc_kmer)
    # determine whether original k-mer or reverse complement is lesser
    if kmer < rc_kmer:
        canonical_kmer = kmer
    else:
        canonical_kmer = rc_kmer

    # calculate murmurhash using a hash seed of 42
    hash = mmh3.hash(canonical_kmer, 42)
    if hash < 0: hash += 2**32

    # done
    return hash

In [11]:
def average_hash(row, size, hashing=32, hash="fnv"):
  kmers = getKmers(row, size)
  #FNV Hash
  if hash == "fnv":
    if hashing == 32:
      hasher = pyhash.fnv1_32()
    if hashing == 64:
      hasher = pyhash.fnv1_64()

    avg = 0
    for kmer in kmers:
      avg += hasher(kmer)
    return avg/(len(row))
  

  #Murmur hash
  if hash == "murmurhash":

    avg = 0
    for kmer in kmers:
      avg += murmur_hash(kmer)
      #print(avg)
    return avg/(len(row))

In [12]:
def apply_hashing(data, all = True, size=1, hashing=32, hash="fnv"):
  #if all == True, include all the possible kmers
  i = 7
  colNames = []
  res = []

  if all:
    for j in range(1, i+1):
      colName = "avg_hash_" + str(j) + "_mer"
      colNames.append(colName)
      res.append(data.apply(lambda x: average_hash(x['nucleo_seq'], size=j, hashing = hashing, hash=hash), axis=1).tolist())
    df = pd.DataFrame(res).transpose()
    df.columns = colNames
    return df
  #else, only the specified k-mer is calculated. Default k-mer is 1
  else:
    colName = "avg_hash_" + str(size) + "_mer"
    colNames.append(colName)    
    res.append(data.apply(lambda x: average_hash(x['nucleo_seq'], size=size, hashing = hashing, hash=hash), axis=1).tolist())
    df = pd.DataFrame(res).transpose()
    df.columns = colNames
    return df

In [13]:
fnv_7mer_hash = apply_hashing(new_df, hash="fnv", all=False, size=7)
fnv_hash_kmer = apply_hashing(new_df, hash="fnv")
mm_hash_kmer = apply_hashing(new_df, hash="murmurhash")

### One-Hot Encoding

In [14]:
base = sorted(set(["A", "T", "C", "G"]))

# integer_encoded = []
# for i in docs:
#     v = np.where( np.array(doc1) == i)[0][0]
#     integer_encoded.append(v)
# print ("\ninteger encoded: ",integer_encoded)

def get_vec(len_doc,word):
    empty_vector = [0] * len_doc
    vect = 0
    find = np.where( np.array(base) == word)[0][0]
    #print(find)
    empty_vector[find] = 1
    return empty_vector

def get_matrix(input):
    mat = []
    len_doc = len(base)
    for i in input:
        vec = get_vec(len_doc,i)
        mat.append(vec)
        
    return np.asarray(mat)

def oneHot(data):
  res = []
  for idx, row in data.iterrows():
    temp = [row["nucleo_seq"]] + list(np.reshape(get_matrix(list(row["nucleo_seq"])),28))
    res.append(temp)
  return res

In [15]:
colName = ["nucleo_seq"]

for j in range(1, 8):
  for i in ["A", "C", "G", "T"]:
    colName.append(i + "_" + str(j))

In [16]:
oneHot_encode = pd.DataFrame(oneHot(new_df), columns=colName).drop(columns=['nucleo_seq'], axis=1)

In [17]:
oneHot_encode

,A_1,C_1,G_1,T_1,A_2,C_2,G_2,T_2,A_3,C_3,...,G_5,T_5,A_6,C_6,G_6,T_6,A_7,C_7,G_7,T_7
0,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,1,0,0,0,0,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,0
121834,1,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,1,0,0,0
121835,0,0,0,1,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
121836,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0


## Getting GGACT Matches

In [18]:
new_df.columns

Index(['transcript_id', 'transcript_pos', 'nucleo_seq', 'dwell_time_-1',
       'sd_-1', 'mean_-1', 'dwell_time_0', 'sd_0', 'mean_0', 'dwell_time_1',
       'sd_1', 'mean_1', 'gene_id', 'transcript_position', 'label'],
      dtype='object')

In [19]:
GGACT = new_df.nucleo_seq.str.match(".GGACT.").astype(int)

## 80-20 Train-Test Split

### We should split the data at a particular index to prevent overlaps in gene_id

In [20]:
def train_test_split(df, labels, train_split):
  # to find out split index for 80-20 split
  init_split = int(train_split * len(df))
  print(f"initial 80% split index: {init_split}")
  last_gene = df.iloc[init_split]["gene_id"]
  print(f"last gene in train set: {last_gene}\n")

  next_gene = last_gene
  split = init_split
  while next_gene==last_gene: # break when next gene different from last gene in train set
    split += 1
    next_gene = df.iloc[split]["gene_id"]

  print(f"final split index (no overlap genes in train/test): {split}")
  print(f"first gene in test set: {next_gene}")
  return df[:split], df[split:], labels[:split], labels[split:], split

In [21]:
# For 80-20 train-test split
X_train, X_test, y_train, y_test, split = train_test_split(new_df, labels, 0.8)

initial 80% split index: 97470
last gene in train set: ENSG00000102804

final split index (no overlap genes in train/test): 97483
first gene in test set: ENSG00000102753


In [22]:
X_test

,transcript_id,transcript_pos,nucleo_seq,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,sd_1,mean_1,gene_id,transcript_position,label
97483,ENST00000261667,3235,TTAACAT,0.006128,1.771679,90.912500,0.006675,2.451071,96.007143,0.005717,2.624607,88.698214,ENSG00000102753,3235,0
97484,ENST00000261667,3006,TAAACTG,0.007165,2.228750,104.394231,0.008992,3.109231,105.186538,0.010010,3.066923,94.971154,ENSG00000102753,3006,0
97485,ENST00000261667,3062,GTAACCA,0.006769,2.558725,88.090196,0.005919,3.134510,94.594118,0.006045,1.899941,82.478431,ENSG00000102753,3062,0
97486,ENST00000261667,3094,CAAACCA,0.009147,2.733273,104.049091,0.008983,4.330000,97.898182,0.005486,2.335455,80.067273,ENSG00000102753,3094,0
97487,ENST00000261667,3197,CAGACAG,0.005387,5.043000,110.800000,0.010036,7.399400,124.840000,0.006664,4.587800,77.220000,ENSG00000102753,3197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000373020,830,ATAACCA,0.007765,2.834000,86.765000,0.007522,3.192500,93.075000,0.005923,2.020500,80.110000,ENSG00000000003,830,0
121834,ENST00000373020,823,ATAACAA,0.007456,2.078095,86.904762,0.006193,1.912857,93.680952,0.007296,2.306667,88.176190,ENSG00000000003,823,0
121835,ENST00000373020,689,TAAACAA,0.009868,1.972857,102.495238,0.007332,2.551905,97.928571,0.006917,2.198571,89.385714,ENSG00000000003,689,0
121836,ENST00000373020,512,ATAACTC,0.007247,2.359500,86.795000,0.011178,2.496500,97.965000,0.008500,1.588750,92.330000,ENSG00000000003,512,0


### Dropping irrelevant columns

## Position Weight Matrix (PWM)

### Creating base probability matrix from training dataset

In [23]:
colSum = oneHot_encode.iloc[:split].sum(axis=0)

In [24]:
def convert_prob_mat(df):
  res=[]
  i=0
  while i < len(list(df)):
    res.append(list(df)[i:i+4])
    i=i+4
  mat = pd.DataFrame(np.array(res).transpose(), columns=['1','2','3','4','5','6','7'])
  mat = mat.rename({0:"A",1:"C",2:"G",3:"T"}, axis="index")

  return mat

In [25]:
mat = convert_prob_mat(colSum)
base_prob_mat = mat.div(split)
base_prob_mat

,1,2,3,4,5,6,7
A,0.319163,0.376620,0.476093,1.0,0.0,0.373870,0.259327
C,0.185407,0.000000,0.000000,0.0,1.0,0.299396,0.210970
G,0.258055,0.345445,0.523907,0.0,0.0,0.000000,0.241427
T,0.237375,0.277936,0.000000,0.0,0.0,0.326734,0.288276


In [26]:
def log_odds(x):
  if x == 0:
    return 0
  else:
    return int(10*np.log10(x/0.25))

In [27]:
log_odds_pos = base_prob_mat.applymap(log_odds)
log_odds_dict = log_odds_pos.to_dict()
log_odds_dict

{'1': {'A': 1, 'C': -1, 'G': 0, 'T': 0},
 '2': {'A': 1, 'C': 0, 'G': 1, 'T': 0},
 '3': {'A': 2, 'C': 0, 'G': 3, 'T': 0},
 '4': {'A': 6, 'C': 0, 'G': 0, 'T': 0},
 '5': {'A': 0, 'C': 6, 'G': 0, 'T': 0},
 '6': {'A': 1, 'C': 0, 'G': 0, 'T': 1},
 '7': {'A': 0, 'C': 0, 'G': 0, 'T': 0}}

In [28]:
def get_PWM(seq):
  res = 0
  for i in range(len(seq)):
    base = seq[i]
    dic = log_odds_dict[str(i+1)]
    res = res + dic[base]
  return res

In [29]:
PWM_col = pd.DataFrame(new_df.apply(lambda x: get_PWM(x["nucleo_seq"]), axis=1), columns=["PWM"])

### Padding engineered features to Dataset

In [30]:
X_train = pd.concat([X_train, mm_hash_kmer, PWM_col, oneHot_encode, GGACT], axis=1).iloc[:split]
X_test = pd.concat([X_test, mm_hash_kmer, PWM_col, oneHot_encode, GGACT], axis=1).iloc[split:]

In [31]:
def val_split(test, test_labels, val_split):

  init_split = int(val_split * len(test))
  print(f"initial 80% split index: {init_split}")
  last_gene = test.iloc[init_split]["gene_id"]
  print(f"last gene in train set: {last_gene}\n")

  next_gene = last_gene
  split = init_split
  while next_gene==last_gene: # break when next gene different from last gene in train set
    split += 1
    next_gene = test.iloc[split]["gene_id"]

  print(f"final split index (no overlap genes in train/validation): {split}")
  print(f"first gene in validation set: {next_gene}")
  return test[:split], test[split:], test_labels[:split], test_labels[split:]

In [32]:
X_test, X_val, y_test, y_val = val_split(X_test, y_test, 0.2)

initial 80% split index: 4871
last gene in train set: ENSG00000099901

final split index (no overlap genes in train/validation): 4912
first gene in validation set: ENSG00000099875


In [33]:
X_test.transcript_pos = X_test.transcript_pos.astype(int)
X_train.transcript_pos = X_train.transcript_pos.astype(int)
X_val.transcript_pos = X_val.transcript_pos.astype(int)

### Dropping unnecessary Columns

In [34]:
X_train.drop(columns=["transcript_id", "transcript_pos", "label", "nucleo_seq", "gene_id"], inplace=True)
X_test.drop(columns=["transcript_id", "transcript_pos", "label", "nucleo_seq", "gene_id"], inplace=True)
X_val.drop(columns=["transcript_id", "transcript_pos", "label", "nucleo_seq", "gene_id"], inplace=True)

In [35]:
X_train["label"] = y_train
X_test["label"] = y_test
X_val["label"] = y_val

### Saving files as CSV

In [36]:
X_train.to_csv("trainset.csv")
X_test.to_csv("testset.csv")
X_val.to_csv("valset.csv")